# Implementation of a graph structure

In [1]:
from random import randint

In [2]:
class Node:
    def __init__(self, ID, mark = False):
        
        self.ID = ID
        self.mark = mark
    
    def __str__(self):
        return self.ID

        
class Edge:
    
    def __init__(self, node1, node2, mark = False):
        
        self.node1 = node1
        self.node2 = node2
        self.mark = mark
    
    def __getitem__(self, key):
        
        if (key == 0):
            return self.node1
        elif (key == 1):
            return self.node2
        
    def __str__(self):
        return "(" + str(self.node1) + "," + str(self.node2) + ")"
    
    def index(self, node):
        if (node == self[0]):
            return 0
        elif (node == self[1]):
            return 1
        

class Graph:
    
    def __init__(self, node_list, edge_list = [], oriented = False):
        
        node_objects = []
        for node in node_list:
            node_objects.append(Node(node))
        
        edge_objects = []
        for edge in edge_list:
            for node in node_objects:
                if(edge[0] == node.ID):
                    node1 = node
                if(edge[1] == node.ID):
                    node2 = node
            edge_objects.append( Edge(node1, node2) )
        
        # Initialise attributes
        self.nodes = node_objects
        self.edges = edge_objects
        self.oriented = oriented
        self.incidence_matrix = self.build_incidence_matrix(node_objects, edge_objects, oriented)
        
    def __str__(self):
        string = "------------Graph------------"
        string += "\n"
        string += "Oriented: " + str(self.oriented)
        string += "\n"
        string += "Nodes: "
        for node in self.nodes:
            string +=" " + str(node)
        string += "\n"
        string += "Edges: "
        for edge in self.edges:
            string += " " + str(edge)
        string += "\n"
        string += "Incidence Matrix: \n"
        for edge in self.incidence_matrix:
            string +="       " + str(edge) + "\n"
        string +="-----------------------------"
        
        return string
        
    def build_incidence_matrix(self, nodes, edges = [], oriented = False):
        # Create incidence matrix
        incidence_matrix = [ [0] * len(nodes) for n in range(len(edges))]

        for edge in edges:
            if (nodes.index(edge[0]) == nodes.index(edge[1])):
                incidence_matrix[ edges.index(edge) ] [ nodes.index(edge[0])] = 2

            else:
                if oriented:
                    incidence_matrix[ edges.index(edge) ] [ nodes.index(edge[0])] = -1
                    incidence_matrix[ edges.index(edge) ] [ nodes.index(edge[1])] = 1
                else:
                    incidence_matrix[ edges.index(edge) ] [ nodes.index(edge[0])] = 1
                    incidence_matrix[ edges.index(edge) ] [ nodes.index(edge[1])] = 1
                    
        return incidence_matrix
    
    def getNode(self, ID):
        for node in self.nodes:
            if node.ID == ID:
                return node
    
    def getEdge(self, ID1, ID2):
        for edge in self.edges:
            if ( (edge[0].ID == ID1 ) & (edge[1].ID == ID2)):
                return edge
    
    def addNode(self, new_node):
        if self.getNode(new_node) in self.nodes:
            print("This node already exists")
        else:
            new_node = Node(new_node)
            self.nodes.append(new_node)
            for edge in self.incidence_matrix:
                edge.append(0)
        
    
    def removeNode(self, node):
        node = self.getNode(node)
        if node in self.nodes:
            valid_edges = []
            for edge in self.edges:
                if ((node != edge[0]) & (node != edge[1])):
                    valid_edges.append(edge)
            self.nodes.remove(node)
            self.edges = valid_edges
            self.incidence_matrix = self.build_incidence_matrix(self.nodes, self.edges, self.oriented)
        else:
            print("This node does not exist")
        
    def connect(self, v1, v2):
        if (self.getNode(v1)!=None) & (self.getNode(v2)!=None):
            new_edge = Edge(self.getNode(v1), self.getNode(v2))
            self.edges.append(new_edge)

            edge = [0] * len(self.nodes)
            if (self.nodes.index(self.getNode(new_edge[0].ID)) == self.nodes.index(self.getNode(new_edge[1].ID))):
                edge[ self.nodes.index(self.getNode(new_edge[0].ID))] = 2

            else:
                if self.oriented:
                    edge[ self.nodes.index(self.getNode(new_edge[0].ID))] = -1
                    edge[ self.nodes.index(self.getNode(new_edge[1].ID))] = 1
                else:
                    edge[ self.nodes.index(self.getNode(new_edge[0].ID))] = 1
                    edge[ self.nodes.index(self.getNode(new_edge[1].ID))] = 1

                self.incidence_matrix.append(edge)
        else:
            print("One of the nodes does not exist")
    
    def disconnect(self, v1, v2):
        edge = self.getEdge(v1, v2)
        if edge in self.edges:
            e = [0] * len(self.nodes)
            if (self.nodes.index(self.getNode(edge[0].ID)) == self.nodes.index(self.getNode(edge[1].ID))):
                e[ self.nodes.index(self.getNode(edge[0].ID))] = 2

            else:
                if self.oriented:
                    e[ self.nodes.index(self.getNode(edge[0].ID))] = -1
                    e[ self.nodes.index(self.getNode(edge[1].ID))] = 1
                else:
                    e[ self.nodes.index(self.getNode(edge[0].ID))] = 1
                    e[ self.nodes.index(self.getNode(edge[1].ID))] = 1
                    
            self.incidence_matrix.remove(e)
            self.edges.remove(edge)
        else:
            print("This edge does not exist")
            
    def order(self):
        return len(self.nodes)
    
    def randomNode(self):
        return self.nodes[randint(0,self.order()-1)]
    
    def adjancentNodes(self, node):
        node = self.getNode(node)
        if node in self.nodes:
            adjacent_nodes = []
            for edge in self.edges:
                if ((node == edge[0]) | (node == edge[1])):
                    if self.oriented:
                        if (node == edge[0]):
                            adjacent_nodes.append(edge[1])
                    else:
                        adjacent_nodes.append(edge[abs(edge.index(node)-1)])
                        
            return adjacent_nodes
        else:
            print("This node does not exist")
            
    def degree(self, node):
        node = self.getNode(node)
        if node in self.nodes:
            return len(self.adjancentNodes(node.ID))
        else:
            print("This node does not exist")
    
    def isRegular(self):
        degree = self.degree(self.randomNode().ID)
        for node in self.nodes:
            if degree != self.degree(node.ID):
                return False
        return True
    
    def isComplete(self):
        for node in self.nodes:
            if len(set(self.adjancentNodes(node.ID))) != (len(self.nodes)-1):
                return False
        return True
    
    def __searchTransitiveClosure(self, node, already_visited):
        already_visited.append(node.ID)
        print(already_visited)
        for adjacent_node in self.adjancentNodes(node.ID):
            if not (adjacent_node.ID in already_visited):
                self.__searchTransitiveClosure(adjacent_node, already_visited)
        return already_visited
        
        
    def transitiveClosure(self, node_id):
        already_visited = []
        node = self.getNode(node_id)
        if node != None:
            return self.__searchTransitiveClosure(node, already_visited)
        else:
            print("This node does not exist")
        
        
            

In [3]:
g = Graph(['a', 'b', 'c'], [('a','b'), ('a','c'), ('a','c'), ('a','c'),('b','a'), ('b', 'c'),('c','a'), ('c','b')], oriented = True)
g.transitiveClosure('a')

['a']
['a', 'b']
['a', 'b', 'c']


['a', 'b', 'c']

In [4]:
print(g)
print("")

print("Connect")
g.connect('c', 'a')
print(g)
print("")

print("Disconnect")
g.disconnect('c', 'a')
print(g)
print("")

print("Add node")
g.addNode('d')
print(g)
print("")

print("Connect")
g.connect('d', 'd')
print(g)
print("")

print("Remove node")
g.removeNode('b')
print(g)
print("")

print("Adjacent nodes")
for node in g.adjancentNodes('a'):
    print(node.ID)
print("")

print("Degree")
print(g.degree('a'))

------------Graph------------
Oriented: True
Nodes:  a b c
Edges:  (a,b) (a,c) (a,c) (a,c) (b,a) (b,c) (c,a) (c,b)
Incidence Matrix: 
       [-1, 1, 0]
       [-1, 0, 1]
       [-1, 0, 1]
       [-1, 0, 1]
       [1, -1, 0]
       [0, -1, 1]
       [1, 0, -1]
       [0, 1, -1]
-----------------------------

Connect
------------Graph------------
Oriented: True
Nodes:  a b c
Edges:  (a,b) (a,c) (a,c) (a,c) (b,a) (b,c) (c,a) (c,b) (c,a)
Incidence Matrix: 
       [-1, 1, 0]
       [-1, 0, 1]
       [-1, 0, 1]
       [-1, 0, 1]
       [1, -1, 0]
       [0, -1, 1]
       [1, 0, -1]
       [0, 1, -1]
       [1, 0, -1]
-----------------------------

Disconnect
------------Graph------------
Oriented: True
Nodes:  a b c
Edges:  (a,b) (a,c) (a,c) (a,c) (b,a) (b,c) (c,b) (c,a)
Incidence Matrix: 
       [-1, 1, 0]
       [-1, 0, 1]
       [-1, 0, 1]
       [-1, 0, 1]
       [1, -1, 0]
       [0, -1, 1]
       [0, 1, -1]
       [1, 0, -1]
-----------------------------

Add node
------------Graph----

In [5]:
print(g)

------------Graph------------
Oriented: True
Nodes:  a c d
Edges:  (a,c) (a,c) (a,c) (c,a) (d,d)
Incidence Matrix: 
       [-1, 1, 0]
       [-1, 1, 0]
       [-1, 1, 0]
       [1, -1, 0]
       [0, 0, 2]
-----------------------------


In [6]:
g.transitiveClosure('a')

['a']
['a', 'c']


['a', 'c']